<a href="https://colab.research.google.com/github/wahyuvlntn/Tweet-Prediction-Bert/blob/main/BERT_WAHYU_SATDAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.37.2
!pip install tensorflow-addons
!pip install tf-keras==2.15.1

In [ ]:
%%capture
!pip install numpy requests nlpaug

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import string

ini cuma contoh

In [ ]:
contoh = {
    'text': [
        "@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= emng bener sih, pendukung 01 ada yg goblok, begitu jg dg pendukung 02.. hnya sj menurut pak Ridwan Kamil skemanya terbalik, klo 01 MAYORITAS PENDIDIKAN MENENGAH ATAS (artinya ada jg pendidikan rendah yg milih",
        "RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7Q= Sewaktu anies bersikap kritis ke kinerja pak prabowo dianggap engga sopan karena dianggap kurang menghormati orang tua, giliran skrg gibran yg tengil dan sok kritis malah dianggap kritis dan keras. Apakah ini tidak standar ganda? [RE coldthem]",
        "RT Duh jangan sampai Pak lurah denger nih ÃƒÂ°Ã‚Å¸Ã‚Â¤Ã‚Â­ Di Acara Hajatan Rakyat, Puluhan Ribu Warga di Kendal Serukan ÃƒÂ¢Ã‚â‚¬Ã‚Å“Ganjar PresidenÃƒÂ¢Ã‚â‚¬Ã‚Â Kehadiran @0Zdeh9QcTWu+z+fS3hRaTcFuSLRh56REFyRLq4/Jdlc= menjadi magnet bagi puluhan ribu warga untuk datang menghadiri Hajatan Rakyat Ganjar-Mahfud. Besarnya antusiasme warga menjadi bukti bahwa Jawa Tengah tetap menjadi kandang banteng. #GanjarMahfud2024 #TabrakProfMahfud [RE DS_yantie]"
    ]
}

In [ ]:
X_train = pd.read_csv('final_train_2.csv')
X_test = pd.read_csv('final_test_2.csv')

In [ ]:
X_train = X_train.sample(frac=1, random_state=42)

In [ ]:
from sklearn.preprocessing import LabelEncoder

X_train['processed_text'] = X_train['processed_text'].astype(str)
X_test['processed_text'] = X_test['processed_text'].astype(str)

X_train['label'] = X_train['label'].astype(str)
X_test['label'] = X_test['label'].astype(str)

label_encoder = LabelEncoder()
X_train['label'] = label_encoder.fit_transform(X_train['label'])
X_test['label'] = label_encoder.fit_transform(X_test['label'])

In [ ]:
from transformers import AutoTokenizer, AutoModel, TFAutoModel, TFAutoModelForSequenceClassification, create_optimizer, AutoConfig, AdamWeightDecay
import tensorflow as tf
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import Sequential

In [ ]:
def clean(text):
    text_cleaning_re = "@\S+|https?:\S+|http?:\S|[#]+|[^A-Za-z0-9]+"
    text = re.sub(text_cleaning_re, " ", text).strip()
    text = text.strip()

    out = []
    for word in text.split():
        out.append(word)

    return " ".join(out).strip()

In [ ]:
X_train["processed_text"] = X_train["processed_text"].apply(clean)
X_test["processed_text"] = X_test["processed_text"].apply(clean)

In [ ]:
%%capture
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-large-p1")
IndoBert = TFAutoModel.from_pretrained("indobenchmark/indobert-large-p1", from_pt=True)

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
tokens_train = tokenizer(
    X_train["processed_text"].tolist(),
    max_length=64,
    padding="max_length",
    truncation=True,
    return_tensors="tf"
)

tokens_val = tokenizer(
    X_test["processed_text"].tolist(),
    max_length=64,
    padding="max_length",
    truncation=True,
    return_tensors="tf"
)

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(X_train.label.tolist(), 8)  # Assuming there are 8 classes
y_val = to_categorical(X_test.label.tolist(), 8)

In [ ]:
y_train.shape

(5135, 8)

In [ ]:
# @title ADASYN

from imblearn.over_sampling import ADASYN
from tensorflow.keras.utils import to_categorical

# Extract input_ids
input_ids_train = tokens_train["input_ids"]

# Apply SMOTE to balance the classes
adasyn = ADASYN(random_state=42)
input_ids_train_resampled, y_train_resampled = adasyn.fit_resample(input_ids_train, X_train['label'].tolist())

# Generate attention masks for resampled data
attention_mask_resampled = np.where(input_ids_train_resampled != tokenizer.pad_token_id, 1, 0)

# Convert labels to categorical format
y_train_resampled_categorical = to_categorical(y_train_resampled, 8)  # Assuming there are 8 classes
y_val = to_categorical(X_test['label'].tolist(), 8)

# Create TensorFlow datasets
train_data_resampled = tf.data.Dataset.from_tensor_slices((
    {"input_ids": input_ids_train_resampled, "attention_mask": attention_mask_resampled},
    y_train_resampled_categorical
)).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

validation_sequence = tf.data.Dataset.from_tensor_slices((
    {"input_ids": tokens_val["input_ids"], "attention_mask": tokens_val["attention_mask"]},
    y_val
)).batch(32).prefetch(tf.data.experimental.AUTOTUNE).cache()

In [ ]:
# @title SMOTE

from imblearn.over_sampling import SMOTE
from tensorflow.keras.utils import to_categorical

# Extract input_ids
input_ids_train = tokens_train["input_ids"]

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
input_ids_train_resampled, y_train_resampled = smote.fit_resample(input_ids_train, X_train['label'].tolist())

# Generate attention masks for resampled data
attention_mask_resampled = np.where(input_ids_train_resampled != tokenizer.pad_token_id, 1, 0)

# Convert labels to categorical format
y_train_resampled_categorical = to_categorical(y_train_resampled, 8)  # Assuming there are 8 classes
y_val = to_categorical(X_test['label'].tolist(), 8)

# Create TensorFlow datasets
train_data_resampled = tf.data.Dataset.from_tensor_slices((
    {"input_ids": input_ids_train_resampled, "attention_mask": attention_mask_resampled},
    y_train_resampled_categorical
)).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

validation_sequence = tf.data.Dataset.from_tensor_slices((
    {"input_ids": tokens_val["input_ids"], "attention_mask": tokens_val["attention_mask"]},
    y_val
)).batch(32).prefetch(tf.data.experimental.AUTOTUNE).cache()

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((
    {"input_ids": tokens_train["input_ids"], "attention_mask": tokens_train["attention_mask"]},
    y_train
)).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

validation_sequence = tf.data.Dataset.from_tensor_slices((
    {"input_ids": tokens_val["input_ids"], "attention_mask": tokens_val["attention_mask"]},
    y_val
)).batch(32).prefetch(tf.data.experimental.AUTOTUNE).cache()

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# classweights
class_weights = compute_class_weight('balanced', classes=np.unique(X_train.label.tolist()), y=X_train.label.tolist())
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
print(class_weights_dict)

#print frekuensi
unique, counts = np.unique(X_train.label.tolist(), return_counts=True)
for class_label, frequency in zip(unique, counts):
    print(f'Class {class_label}: {frequency}')


{0: 2.5270669291338583, 1: 2.090798045602606, 2: 4.114583333333333, 3: 1.3099489795918366, 4: 1.3744646680942185, 5: 0.2691299790356394, 6: 0.8964734636871509, 7: 1.7829861111111112}
Class 0: 254
Class 1: 307
Class 2: 156
Class 3: 490
Class 4: 467
Class 5: 2385
Class 6: 716
Class 7: 360


In [ ]:
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [ ]:
print(label_encoder.classes_)

['Demografi' 'Ekonomi' 'Geografi' 'Ideologi' 'Pertahanan dan Keamanan'
 'Politik' 'Sosial Budaya' 'Sumber Daya Alam']


In [ ]:
# @title LDAM Loss

class LDAMLoss(tf.keras.losses.Loss):
    def __init__(self, cls_num_list, max_m=0.5, s=30):
        super(LDAMLoss, self).__init__()
        m_list = 1.0 / np.sqrt(np.sqrt(cls_num_list))
        m_list = m_list * (max_m / np.max(m_list))
        self.m_list = tf.convert_to_tensor(m_list, dtype=tf.float32)
        self.s = s

    def call(self, y_true, y_pred):
        # Ensure y_true and y_pred have the same shape
        if y_true.shape != y_pred.shape:
            raise ValueError("Shapes of y_true and y_pred must match")

        # Calculate margins for each class based on the true labels
        margins = tf.reduce_sum(y_true * self.m_list, axis=-1, keepdims=True)

        # Apply the margins to the predictions
        y_pred_m = y_pred - margins

        # Scale the logits
        y_pred_scaled = self.s * y_pred_m

        # Calculate the loss
        return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred_scaled))


# import tensorflow_model_analysis as tfma
num_classes = 8  # Number of classes in your dataset
cls_num_list = [49, 308, 16, 319, 304, 2386, 467, 151]

# Create a custom model architecture using IndoBert as a base
input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

embeddings = IndoBert(input_ids, attention_mask=attention_mask)[0]
pooled_output = tf.keras.layers.GlobalAveragePooling1D()(embeddings)
dropout = tf.keras.layers.Dropout(0.1)(pooled_output)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(dropout)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=LDAMLoss(cls_num_list=cls_num_list),
              metrics=["accuracy"])

In [ ]:
# @title Simple
num_classes = 8

# for layer in IndoBert.layers:
#     layer.trainable = False

# Create a custom model architecture using IndoBert as a base
input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

bert_outputs = IndoBert(input_ids, attention_mask=attention_mask)
pooled_output = bert_outputs.pooler_output
bert_outputs.trainable = False

dense_layer = tf.keras.layers.Dense(512, activation='relu')(pooled_output)
dense_layer = tf.keras.layers.Dense(512, activation='relu')(pooled_output)
dropout = tf.keras.layers.Dropout(0.2)(dense_layer)
dense_layer = tf.keras.layers.Dense(1024, activation='relu')(dropout)

outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(dense_layer)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# @title Simple 2 Use last_hidden_state
num_classes = 8

for layer in IndoBert.layers:
    layer.trainable = False

# Create a custom model architecture using IndoBert as a base
input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

bert_outputs = IndoBert(input_ids, attention_mask=attention_mask)
last_hidden_state = bert_outputs.last_hidden_state[:, 0, :]

"""
use global average pooling
last_hidden_state = bert_outputs.last_hidden_state
avg_pool = tf.keras.layers.GlobalAveragePooling1D()(last_hidden_state)

"""

# dropout = tf.keras.layers.Dropout(0.2)(last_hidden_state)
dense_layer = tf.keras.layers.Dense(512, activation='relu')(last_hidden_state)

outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(dense_layer)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# # @title LSTM model
from transformers import TFBertModel
class BERTLSTMClassifier(tf.keras.Model):
    def __init__(self, num_classes, hidden_size):
        super(BERTLSTMClassifier, self).__init__()
        self.bert = TFBertModel.from_pretrained('indobenchmark/indobert-large-p1')
        self.lstm = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.dense1 = tf.keras.layers.Dense(hidden_size, activation='relu')
        self.batch_norm = tf.keras.layers.BatchNormalization()
        self.dense2 = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        lstm_output = self.lstm(outputs.last_hidden_state)
        # mean_pooling = tf.reduce_mean(lstm_output, axis=1)  # Mean pooling
        max_pooling = tf.reduce_max(lstm_output, axis=1)  # max pooling
        x = self.dense1(max_pooling)
        x = self.batch_norm(x)
        logits = self.dense2(x)
        return logits

input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name='attention_mask')
outputs = BERTLSTMClassifier(num_classes=8, hidden_size=768)((input_ids, attention_mask))

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tf_model.h5:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

Some layers from the model checkpoint at indobenchmark/indobert-large-p1 were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at indobenchmark/indobert-large-p1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
from transformers import TFBertModel

class BERTMeanPoolingClassifier(tf.keras.Model):
    def __init__(self, num_classes, hidden_size):
        super(BERTMeanPoolingClassifier, self).__init__()
        self.bert = TFBertModel.from_pretrained('indobenchmark/indobert-large-p1')
        self.dense1 = tf.keras.layers.Dense(hidden_size, activation='relu')
        # self.dropout = tf.keras.layers.Dropout(0.3)
        self.dense12 = tf.keras.layers.Dense(hidden_size // 2, activation='relu')
        self.batch_norm = tf.keras.layers.BatchNormalization()
        self.dense2 = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        mean_pooling = tf.reduce_mean(outputs.last_hidden_state, axis=1)  # Mean pooling
        x = self.dense1(mean_pooling)
        # x = self.dropout(x)
        x = self.dense12(x)
        x = self.batch_norm(x)
        logits = self.dense2(x)
        return logits


input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name='attention_mask')
outputs = BERTMeanPoolingClassifier(num_classes=8, hidden_size=768)((input_ids, attention_mask))
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some layers from the model checkpoint at indobenchmark/indobert-large-p1 were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at indobenchmark/indobert-large-p1.
If yo

In [ ]:
class LearningRateScheduler(tf.keras.callbacks.Callback):
    def __init__(self, initial_lr, factor, epochs_per_decay):
        super(LearningRateScheduler, self).__init__()
        self.initial_lr = initial_lr
        self.factor = factor
        self.epochs_per_decay = epochs_per_decay

    def on_epoch_begin(self, epoch, logs=None):
        if (epoch + 1) % self.epochs_per_decay == 0:
            lr = self.initial_lr * (self.factor ** ((epoch + 1) // self.epochs_per_decay))
            tf.keras.backend.set_value(self.model.optimizer.lr, lr)
            print(f'Learning rate adjusted to {lr} for epoch {epoch + 1}')

callbacks = [
    # tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3),
    LearningRateScheduler(initial_lr=2e-5, factor=0.1, epochs_per_decay=5)
    # tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=2, min_lr=1e-8)
]

model.fit(
    train_data,
    validation_data=validation_sequence,
    epochs=3,
    batch_size=32,
    class_weight=class_weights_dict,
    callbacks=callbacks
)

Epoch 1/3
161/161 [==============================] - 103s 643ms/step - loss: 0.0708 - accuracy: 0.9673 - val_loss: 0.9868 - val_accuracy: 0.7490
Epoch 2/3
161/161 [==============================] - 103s 643ms/step - loss: 0.0643 - accuracy: 0.9696 - val_loss: 1.0025 - val_accuracy: 0.7420
Epoch 3/3
161/161 [==============================] - 103s 642ms/step - loss: 0.0599 - accuracy: 0.9722 - val_loss: 1.0259 - val_accuracy: 0.7420


# HASIL BIASA

In [ ]:
# predict training data
y_pred_train = model.predict(train_data)
y_pred_train = np.argmax(y_pred_train, axis=1)

# one-hot encoded y_train to original
y_train_original = np.argmax(y_train, axis=1)

# print
print("Training - Predicted values:")
print(y_pred_train)

print("Training - Actual values:")
print(y_train_original)

# predict validation data
y_pred_val = model.predict(validation_sequence)
y_pred_val = np.argmax(y_pred_val, axis=1)

# one-hot encoded y_val to original
y_val_original = np.argmax(y_val, axis=1)

# print
print("Validation - Predicted values:")
print(y_pred_val)

print("Validation - Actual values:")
print(y_val_original)

161/161 [==============================] - 29s 183ms/step
Training - Predicted values:
[3 5 5 ... 5 4 5]
Training - Actual values:
[3 5 5 ... 5 4 5]
32/32 [==============================] - 6s 177ms/step
Validation - Predicted values:
[5 4 4 5 5 6 5 5 5 6 1 3 6 5 6 7 5 5 4 3 0 4 5 1 5 3 4 4 5 1 6 4 6 5 5 5 5
 5 5 1 1 5 5 5 5 3 6 5 5 5 5 3 5 5 3 1 6 5 5 3 4 7 6 4 5 4 5 4 5 5 3 5 4 1
 7 5 5 5 5 6 1 5 5 1 6 5 5 5 5 6 3 5 0 6 4 5 5 6 4 5 6 5 5 5 5 4 3 5 5 6 5
 6 5 5 0 5 5 0 6 0 0 5 5 6 7 5 5 5 5 4 6 5 6 5 6 4 3 5 5 5 5 5 4 5 5 5 5 1
 0 5 3 6 5 4 7 6 1 6 5 5 5 5 5 1 4 4 5 6 3 5 6 4 4 6 3 5 5 5 5 4 5 5 5 5 1
 5 5 6 5 5 4 5 6 5 6 4 5 5 5 6 2 3 5 4 5 5 5 5 1 4 5 5 3 6 6 0 5 5 5 5 5 6
 6 5 5 1 5 5 5 5 4 5 5 5 5 5 6 5 5 3 5 5 3 3 5 7 7 3 4 5 4 3 4 5 5 4 5 5 1
 6 5 5 5 1 5 1 4 5 5 5 5 4 6 4 5 5 4 1 5 5 5 3 5 3 3 3 5 3 5 5 5 5 6 5 5 5
 5 7 5 5 6 5 6 6 5 5 3 5 5 5 5 5 6 5 5 5 5 4 6 5 5 5 0 3 5 5 5 4 5 5 5 6 5
 7 5 7 5 7 6 6 4 4 5 3 3 5 5 5 5 5 5 4 5 3 4 5 5 4 1 5 5 5 5 3 5 3 5 5 7 5
 5 5 4 3 5 4 5 

In [ ]:
#[CLS] Token Representation
from sklearn.metrics import classification_report, balanced_accuracy_score

# Calculate balanced accuracy for training data
balanced_acc_train = balanced_accuracy_score(y_train_original, y_pred_train)

# Calculate balanced accuracy for validation data
balanced_acc_val = balanced_accuracy_score(y_val_original, y_pred_val)

# Print the balanced accuracy
print("Training - Balanced Accuracy:", balanced_acc_train)
print("Validation - Balanced Accuracy:", balanced_acc_val)

# Print classification report for training data
print("Training Classification Report:")
print(classification_report(y_train_original, y_pred_train))

# Print classification report for validation data
print("Validation Classification Report:")
print(classification_report(y_val_original, y_pred_val))

#

Training - Balanced Accuracy: 0.9907094276844064
Validation - Balanced Accuracy: 0.6003847945538725
Training Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       254
           1       0.92      1.00      0.96       307
           2       1.00      1.00      1.00       156
           3       0.96      0.99      0.98       490
           4       0.97      0.99      0.98       467
           5       1.00      0.98      0.99      2385
           6       0.99      0.98      0.98       716
           7       0.98      0.99      0.98       360

    accuracy                           0.98      5135
   macro avg       0.98      0.99      0.98      5135
weighted avg       0.98      0.98      0.98      5135

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.19      0.23      0.21        13
           1       0.59      0.68      0.63        59
           2       0

# HASIL SMOTE

In [ ]:
# predict training data
y_pred_train = model.predict(train_data)
y_pred_train = np.argmax(y_pred_train, axis=1)

# one-hot encoded y_train to original
y_train_original = np.argmax(y_train, axis=1)

# print
print("Training - Predicted values:")
print(y_pred_train)

print("Training - Actual values:")
print(y_train_original)

# predict validation data
y_pred_val = model.predict(validation_sequence)
y_pred_val = np.argmax(y_pred_val, axis=1)

# one-hot encoded y_val to original
y_val_original = np.argmax(y_val, axis=1)

# print
print("Validation - Predicted values:")
print(y_pred_val)

print("Validation - Actual values:")
print(y_val_original)

161/161 [==============================] - 29s 178ms/step
Training - Predicted values:
[3 5 5 ... 5 4 5]
Training - Actual values:
[3 5 5 ... 5 4 5]
32/32 [==============================] - 6s 174ms/step
Validation - Predicted values:
[5 4 4 5 5 6 5 5 5 6 1 5 1 5 6 7 5 5 4 3 5 4 5 1 5 3 4 4 5 1 3 4 6 5 5 5 5
 5 5 1 1 5 5 6 5 3 6 5 5 5 5 3 5 5 3 1 6 5 5 3 4 6 6 4 5 6 5 4 5 5 3 5 4 1
 7 5 5 5 5 6 1 5 5 4 6 5 5 5 5 6 3 5 5 6 4 5 5 6 4 5 5 5 5 3 5 4 3 5 5 6 1
 6 5 5 0 5 5 0 6 4 0 5 6 6 7 5 5 5 5 4 6 5 6 5 6 4 3 5 5 5 2 5 6 5 5 5 6 1
 1 5 3 6 5 4 7 6 1 6 5 5 5 5 5 6 4 4 5 6 3 5 6 4 4 6 6 5 5 5 5 4 5 5 5 5 1
 0 5 6 5 4 4 5 6 5 5 4 5 4 6 6 5 3 4 4 5 5 5 5 1 4 5 5 5 6 6 0 5 5 4 5 5 6
 4 5 5 6 5 5 5 5 4 5 5 4 6 5 6 5 5 3 5 5 3 3 5 5 6 3 4 5 4 6 4 5 5 4 1 5 1
 6 4 5 6 1 6 7 4 5 5 5 5 4 6 4 5 5 4 0 5 5 5 6 5 4 6 5 5 3 5 5 5 5 6 5 5 5
 5 7 5 5 6 5 6 6 5 5 3 5 5 5 5 5 1 5 5 5 5 4 6 5 5 5 6 3 5 5 5 4 6 5 5 6 5
 5 6 7 6 3 6 5 4 4 5 3 6 5 5 6 5 5 3 4 5 5 4 5 4 4 1 5 5 3 5 3 5 3 5 5 7 5
 5 4 4 5 5 4 5 

In [ ]:
#[CLS] Token Representation
from sklearn.metrics import classification_report, balanced_accuracy_score

# Calculate balanced accuracy for training data
balanced_acc_train = balanced_accuracy_score(y_train_original, y_pred_train)

# Calculate balanced accuracy for validation data
balanced_acc_val = balanced_accuracy_score(y_val_original, y_pred_val)

# Print the balanced accuracy
print("Training - Balanced Accuracy:", balanced_acc_train)
print("Validation - Balanced Accuracy:", balanced_acc_val)

# Print classification report for training data
print("Training Classification Report:")
print(classification_report(y_train_original, y_pred_train))

# Print classification report for validation data
print("Validation Classification Report:")
print(classification_report(y_val_original, y_pred_val))

#

Training - Balanced Accuracy: 0.9903553347606822
Validation - Balanced Accuracy: 0.5989940987228244
Training Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       254
           1       0.93      0.99      0.96       307
           2       0.99      1.00      1.00       156
           3       0.99      0.99      0.99       490
           4       0.95      0.99      0.97       467
           5       1.00      0.98      0.99      2385
           6       0.98      0.99      0.98       716
           7       0.99      0.99      0.99       360

    accuracy                           0.98      5135
   macro avg       0.98      0.99      0.98      5135
weighted avg       0.99      0.98      0.98      5135

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.36      0.38      0.37        13
           1       0.55      0.71      0.62        59
           2       0

# HASIL ADASYN

In [ ]:
# predict training data
y_pred_train = model.predict(train_data)
y_pred_train = np.argmax(y_pred_train, axis=1)

# one-hot encoded y_train to original
y_train_original = np.argmax(y_train, axis=1)

# print
print("Training - Predicted values:")
print(y_pred_train)

print("Training - Actual values:")
print(y_train_original)

# predict validation data
y_pred_val = model.predict(validation_sequence)
y_pred_val = np.argmax(y_pred_val, axis=1)

# one-hot encoded y_val to original
y_val_original = np.argmax(y_val, axis=1)

# print
print("Validation - Predicted values:")
print(y_pred_val)

print("Validation - Actual values:")
print(y_val_original)

161/161 [==============================] - 34s 177ms/step
Training - Predicted values:
[3 6 5 ... 5 4 5]
Training - Actual values:
[3 5 5 ... 5 4 5]
32/32 [==============================] - 6s 174ms/step
Validation - Predicted values:
[5 4 4 5 6 6 5 5 5 6 1 6 6 5 6 7 5 5 4 3 3 4 5 1 5 3 4 5 5 1 3 4 6 5 5 5 5
 5 5 1 1 5 5 6 5 3 6 5 5 5 6 3 5 5 3 1 6 5 5 3 4 6 6 4 5 6 5 4 5 5 3 5 4 5
 7 5 5 5 5 6 1 5 5 4 6 5 5 5 5 6 3 5 6 6 4 5 5 6 4 5 5 5 5 3 5 4 3 5 5 6 5
 6 4 5 0 3 5 0 6 0 0 5 0 7 7 5 6 5 5 4 6 5 0 5 6 4 3 5 5 5 6 6 4 6 5 5 5 1
 1 5 3 6 5 4 7 6 1 6 5 5 0 1 5 5 4 4 5 6 3 5 6 4 4 6 6 5 5 5 5 4 5 5 5 5 1
 5 5 6 5 4 4 5 6 5 6 4 5 5 5 6 1 3 4 4 5 5 5 5 1 4 5 5 5 6 6 6 5 5 4 5 5 6
 6 5 5 1 5 5 5 5 4 5 5 5 5 5 6 5 5 3 5 0 3 3 5 5 6 3 4 5 4 6 4 5 5 4 5 5 1
 6 7 3 6 1 6 6 4 1 5 5 1 4 6 4 5 5 4 3 5 5 5 6 5 4 6 5 5 3 5 5 5 5 6 5 1 5
 5 7 5 5 6 5 6 6 5 5 3 6 3 5 5 5 6 5 5 5 5 4 6 5 5 5 0 3 5 5 5 4 5 5 5 6 5
 3 6 7 6 7 6 5 4 4 5 3 3 5 5 6 5 5 5 4 5 1 4 5 5 4 1 5 5 3 5 3 1 3 5 5 7 5
 5 4 4 5 5 4 5 

In [ ]:
#[CLS] Token Representation
from sklearn.metrics import classification_report, balanced_accuracy_score

# Calculate balanced accuracy for training data
balanced_acc_train = balanced_accuracy_score(y_train_original, y_pred_train)

# Calculate balanced accuracy for validation data
balanced_acc_val = balanced_accuracy_score(y_val_original, y_pred_val)

# Print the balanced accuracy
print("Training - Balanced Accuracy:", balanced_acc_train)
print("Validation - Balanced Accuracy:", balanced_acc_val)

# Print classification report for training data
print("Training Classification Report:")
print(classification_report(y_train_original, y_pred_train))

# Print classification report for validation data
print("Validation Classification Report:")
print(classification_report(y_val_original, y_pred_val))

#

Training - Balanced Accuracy: 0.9812325483578208
Validation - Balanced Accuracy: 0.6220487864101887
Training Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       254
           1       0.84      0.98      0.90       307
           2       1.00      1.00      1.00       156
           3       0.92      0.99      0.95       490
           4       0.94      0.99      0.96       467
           5       1.00      0.92      0.96      2385
           6       0.92      0.98      0.95       716
           7       0.94      1.00      0.97       360

    accuracy                           0.96      5135
   macro avg       0.94      0.98      0.96      5135
weighted avg       0.96      0.96      0.96      5135

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.27      0.31      0.29        13
           1       0.55      0.78      0.65        59
           2       0